In [1]:
import urllib3
import certifi
import datetime
import os
import pandas as pd
from time import sleep
from bs4 import BeautifulSoup

In [2]:
data_dir = 'ScrappedData'
fname = 'WalkUpMusic.csv'
overwrite = True
# Get team IDs for URL queries
teams_df = pd.read_csv('baseballdatabank/core/Teams.csv')
teams2_df = teams_df[teams_df.yearID == 
                     max(teams_df.yearID)].filter(items = 
                                                  ['name', 
                                                   'teamIDBR'])
# Some of the shorthands don't match, so correct by hand
teams2_df.replace(['CHW','KCR','SDP','SFG','TBR', 'WSN'], 
                  ['CWS','KC','SD', 'SF','TB', 'WSH'])

,name,teamIDBR
2835,Arizona Diamondbacks,ARI
2836,Atlanta Braves,ATL
2837,Baltimore Orioles,BAL
2838,Boston Red Sox,BOS
2839,Chicago White Sox,CWS
2840,Chicago Cubs,CHC
2841,Cincinnati Reds,CIN
2842,Cleveland Indians,CLE
2843,Colorado Rockies,COL
2844,Detroit Tigers,DET


In [3]:
# Query MLB.com, each team has it's own walk up music page
d = []
url_prefix ='https://www.mlb.com/entertainment/walk-up/'
for idx, team in teams2_df.iterrows():
    print(team)
    url = url_prefix + team['teamIDBR']
    http = urllib3.PoolManager(cert_reqs='CERT_REQUIRED',ca_certs=certifi.where())
    r = http.request('GET', url)
    sleep(2)
    html_decode = r.data.decode('utf-8')
    soup = BeautifulSoup(html_decode, 'html.parser')
    table_rows = soup.findAll('tr')
    for idx, row in enumerate(table_rows):
        name = []
        song_artist=[]
        song_name=[]
        if row.find('p', class_="player-name") is not None:
            name = row.find('p', class_="player-name").string.strip()
            if ' - ' in name:
                name = name.split(' - ')[0]
            # HTML is inconsistently written, 
            #usually span has a class but sometimes...
            if row.find('span', class_="song-artist") is not None:
                song_artist = row.find('span', class_="song-artist").string
                song_title = row.find('span', class_="song-title").string
                lang = 'en' # english unless otherwise specified
            # Song title/artist is in one tag, and that needs to be parsed
            else:
                div = row.find('div', class_="song-name")
                if len(div.findAll('span')) == 0: 
                    # If nothing in the span, no walk up music
                    song_artist = None
                    song_title = None
                    lang = None  
                else:
                    song_artist = div.find('span').string
                    if len(div.findAll('span')) == 2:
                        # If there are two spans with song & artist in each
                        song_title = row.findAll('span')[1].string
                        lang = row.findAll('span')[1]['lang'].lower()
                    else:
                        # If there is only one span with artist outside of span
                        div_sib = div.find('span').next_sibling
                        lang = 'en'
                        if div_sib is not None:
                            song_title = div_sib.string.strip()
                        else: 
                            song_title = song_artist
                            song_artist = None

            d.append({'Team_Name': team['name'],
                      'Team_ID': team['teamIDBR'],
                      'Player_Name': name, 
                      'Song_Artist': song_artist, 
                      'Song_Title': song_title, 
                      'Language': lang,
                      'Date_Updated': datetime.datetime.today().date()})

name        Arizona Diamondbacks
teamIDBR                     ARI
Name: 2835, dtype: object
name        Atlanta Braves
teamIDBR               ATL
Name: 2836, dtype: object
name        Baltimore Orioles
teamIDBR                  BAL
Name: 2837, dtype: object
name        Boston Red Sox
teamIDBR               BOS
Name: 2838, dtype: object
name        Chicago White Sox
teamIDBR                  CHW
Name: 2839, dtype: object
name        Chicago Cubs
teamIDBR             CHC
Name: 2840, dtype: object
name        Cincinnati Reds
teamIDBR                CIN
Name: 2841, dtype: object
name        Cleveland Indians
teamIDBR                  CLE
Name: 2842, dtype: object
name        Colorado Rockies
teamIDBR                 COL
Name: 2843, dtype: object
name        Detroit Tigers
teamIDBR               DET
Name: 2844, dtype: object
name        Houston Astros
teamIDBR               HOU
Name: 2845, dtype: object
name        Kansas City Royals
teamIDBR                   KCR
Name: 2846, dtype: object


In [4]:
data_path = data_dir + '/' + fname
if not os.path.isfile(data_path) or overwrite:            
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)
    pd.DataFrame(d).to_csv(data_path, index=False)
else: 
    old_d = pd.read_csv(data_path)
    old_d.append(d).to_csv(data_path, index=False)